In [1]:
import QUANTAXIS as QA

QUANTAXIS>> start QUANTAXIS
QUANTAXIS>> Welcome to QUANTAXIS, the Version is 1.1.4.dev2
QUANTAXIS>>  
 ```````````````````````````````````````````````````````````````````````````````````````````````````````````````````````` 
  ``########`````##````````##``````````##`````````####````````##```##########````````#``````##``````###```##`````######`` 
  `##``````## ```##````````##`````````####````````##`##```````##```````##```````````###``````##````##`````##```##`````##` 
  ##````````##```##````````##````````##`##````````##``##``````##```````##``````````####```````#```##``````##```##``````## 
  ##````````##```##````````##```````##```##```````##```##`````##```````##`````````##`##```````##`##```````##````##``````` 
  ##````````##```##````````##``````##`````##``````##````##````##```````##````````##``###```````###````````##`````##`````` 
  ##````````##```##````````##``````##``````##`````##`````##```##```````##```````##````##```````###````````##``````###```` 
  ##````````##```##````````##`````##`

In [2]:
# 股票Account

In [3]:
# 期货Account

In [4]:
rb=QA.QA_fetch_get_future_min('tdx','RBL8','2018-01-01','2018-02-28',frequence='15min')
rb_ds=QA.QA_DataStruct_Future_min(rb.set_index(['datetime','code']))

AccountFuture=QA.QA_Account(init_cash=1000000,allow_sellopen=True,allow_t0=True,account_cookie='future_test',market_type=QA.MARKET_TYPE.FUTURE_CN,frequence=QA.FREQUENCE.FIFTEEN_MIN)


#Account.reset_assets(10000000)
Broker=QA.QA_BacktestBroker()

QUANTAXIS>> Selecting the Best Server IP of TDX


USING DEFAULT STOCK IP
USING DEFAULT FUTURE IP


QUANTAXIS>> === The BEST SERVER ===
 stock_ip 115.238.56.198 future_ip 106.14.95.149


In [5]:
hq=next(rb_ds.panel_gen)

In [6]:
AccountFuture.cash_available

1000000

# 1. 先卖空开仓的场景

In [7]:

order=AccountFuture.send_order(code='RBL8',
                        price=hq.open,
                        money=AccountFuture.cash_available,
                        time=hq.datetime[0],
                        towards=QA.ORDER_DIRECTION.SELL_OPEN,
                        order_model=QA.ORDER_MODEL.MARKET,
                        amount_model=QA.AMOUNT_MODEL.BY_MONEY
                        )


In [8]:
Broker.receive_order(QA.QA_Event(order=order,market_data=hq))

< QA_Order realorder_id Order_Fzi9LqH1 datetime:2018-01-02 09:15:00 code:RBL8 amount:200 price:3801.5 towards:-2 btype:future_cn order_id:Order_Fzi9LqH1 account:future_test status:queued >

In [9]:
trade_mes = Broker.query_orders(AccountFuture.account_cookie, 'filled')
res = trade_mes.loc[order.account_cookie, order.realorder_id]
order.trade(res.trade_id, res.trade_price, res.trade_amount, res.trade_time)

receive deal


In [10]:
AccountFuture.allow_t0

True

In [11]:
AccountFuture.hold

code
RBL8   -200
Name: amount, dtype: int64

In [12]:
AccountFuture.time_index

['2018-01-02 09:15:00']

In [13]:
AccountFuture.history

[['2018-01-02 09:15:00',
  'RBL8',
  3801.5,
  -200,
  239890.07499999995,
  'Order_Fzi9LqH1',
  'Order_Fzi9LqH1',
  'Trade_rsaKZUmh',
  'future_test',
  190.075,
  0]]

In [14]:
AccountFuture.sell_available

code
RBL8   -200
Name: amount, dtype: int64

In [15]:
AccountFuture.buy_available

code
RBL8   -200
Name: amount, dtype: int64

In [16]:
AccountFuture.history_table

,datetime,code,price,amount,cash,order_id,realorder_id,trade_id,account_cookie,commission,tax
0,2018-01-02 09:15:00,RBL8,3801.5,-200,239890.075,Order_Fzi9LqH1,Order_Fzi9LqH1,Trade_rsaKZUmh,future_test,190.075,0


In [17]:
AccountFuture.cash_available

239890.07499999995

In [18]:
AccountFuture.hold

code
RBL8   -200
Name: amount, dtype: int64

In [19]:
AccountFuture.hold_available

code
RBL8   -200
Name: amount, dtype: int64

In [20]:
AccountFuture.daily_hold

,code,RBL8
date,account_cookie,
2018-01-02,future_test,-200


In [21]:
AccountFuture.cash

[1000000, 239890.07499999995]

In [22]:
order

< QA_Order realorder_id Order_Fzi9LqH1 datetime:2018-01-02 09:15:00 code:RBL8 amount:200 price:3801.5 towards:-2 btype:future_cn order_id:Order_Fzi9LqH1 account:future_test status:success_all >

In [23]:
rbd=QA.QA_fetch_get_future_day('tdx','RBL8','2018-01-01','2018-02-28')
rbd_ds=QA.QA_DataStruct_Future_day(rbd.set_index(['date','code']),dtype='future_day')

In [24]:
#rbd_ds.data.reset_index()#

In [25]:
AccountFuture.history_table

,datetime,code,price,amount,cash,order_id,realorder_id,trade_id,account_cookie,commission,tax
0,2018-01-02 09:15:00,RBL8,3801.5,-200,239890.075,Order_Fzi9LqH1,Order_Fzi9LqH1,Trade_rsaKZUmh,future_test,190.075,0


In [26]:
Risk=QA.QA_Risk(AccountFuture,if_fq=False,market_data=rbd_ds)

In [27]:
Risk.daily_market_value

date        account_cookie
2018-01-02  future_test       774800.0
dtype: float64

In [28]:
Risk.account.daily_hold

,code,RBL8
date,account_cookie,
2018-01-02,future_test,-200


In [29]:
Risk.account.daily_cash

,,cash,datetime,date,account_cookie
date,account_cookie,,,,
2018-01-02,future_test,239890,2018-01-02 09:15:00,2018-01-02,future_test


In [30]:
Risk.assets

date
2018-01-02    1014690.075
Name: 0, dtype: float64

# 2. 继续卖空 但是账户资金不够

In [31]:

order=AccountFuture.send_order(code='RBL8',
                        price=hq.open,
                        money=2*AccountFuture.cash_available,
                        time=hq.datetime[0],
                        towards=QA.ORDER_DIRECTION.SELL_OPEN,
                        order_model=QA.ORDER_MODEL.MARKET,
                        amount_model=QA.AMOUNT_MODEL.BY_MONEY
                        )


sellavailable -200
amount 100
aqureMoney 479780.1499999999
cash 239890.07499999995
卖空资金不足/不允许裸卖空
ERROR : CODE RBL8 TIME 2018-01-02 09:15:00  AMOUNT 100 TOWARDS -2


## 3. SETTLE事件